In [ ]:
import quetchup as Q

ModuleNotFoundError: No module named 'quetchup'

## Working with States

Qubos defines two main classes: `State` and `Map`. We'll start by looking at the `State` class and what it can do. Let's first define a `State` object consisting of 3 qubits, all in the ground state (i.e. the state $|000\rangle$).

In [ ]:
my_qubits = Q.State(3)
my_qubits

If you want to visualize the state, you can use the `.to_latex()` function:

In [ ]:
Q.to_latex(my_qubits)

<IPython.core.display.Math object>

If you forget how many qubits are in the state, you can use the `.n()` method:

In [ ]:
my_qubits.n()

3

One can also apply any Clifford unitaries to the state:
* `apply_h(state, pos)` applies a Hadamard to the qubit at position `pos`
* `apply_cx(state, pos0, pos1)` applies a CNOT with the qubit at position `pos0` as control and the qubit at position `pos1` as target
* `apply_s(state, pos)` applies an S gate to the qubit at position `pos`
* `apply_swap(state, pos0, pos1)` applies a SWAP gate with the qubit at position `pos0` and the qubit at position `pos1`
* `apply_x(state, pos)` applies an X gate to the qubit at position `pos`
* `apply_z(state, pos)` applies a Z gate to the qubit at position `pos`


In [ ]:
Q.apply_h(my_qubits, 0)     # applies a Hadamard to the first qubit
Q.apply_cx(my_qubits, 0, 1) # applies a CNOT with the first qubit as control and the second as target
Q.apply_cx(my_qubits, 1, 2) # applies a CNOT with the second qubit as control and the third as target
Q.apply_s(my_qubits, 0)     # applies an S gate to the first qubit
Q.to_latex(my_qubits)

<IPython.core.display.Math object>

Checking equality of states is as simple as:

In [ ]:
state0 = Q.State(2)
Q.apply_h(state0, 0)
Q.apply_cx(state0, 0, 1)


state1 = Q.State(2)
Q.apply_h(state1, 1)
Q.apply_cx(state1, 1, 0)

[state0 == state1, state0 == my_qubits]

[True, False]

Part of the innovation of Qubos is that one can also append qubits and postselect on them while still accurately keeping track of the global phase. For example, the following code starts with a scalar state, appends a qubit, applies a Hadamard and then postselects on the first qubit being in the state $|0\rangle$ - i.e. it computes the operation $\langle 0 | H | 0 \rangle$.

In [ ]:
my_qubits = Q.State()
my_qubits.zero_prep()
Q.apply_h(my_qubits, 0)
my_qubits.zero_postselect(0) # postselects on the first qubit (on position 0) being in the state $|0\rangle$

Q.to_latex(my_qubits)

<IPython.core.display.Math object>

One can insert new qubits or postselect at arbitrary positions:

In [ ]:
my_qubits = Q.State()
for i in range(3):
    my_qubits.zero_prep(0)
    Q.apply_x(my_qubits, 0)

display(Q.to_latex(my_qubits)) # |111>

my_qubits.zero_prep(1)
display(Q.to_latex(my_qubits)) # |1011>

my_qubits.zero_postselect(1)
display(Q.to_latex(my_qubits)) # |111>

my_qubits.zero_postselect(1)
display(Q.to_latex(my_qubits)) # 0

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## The Insides of a State

Any non-zero stabilizer state $| \psi \rangle$ has the property that it can be written as follows:

$$
| \psi \rangle = e^\frac{2 p i \pi}{8} \frac{1}{\sqrt{2^m}} \sum_{A \vec{x} = \vec{b}} i^{\vec{x} \cdot \ell + \vec{x}^\intercal Q \vec{x} } | x \rangle
$$

where:
* $A$ and $b$ are a boolean matrix and a vector of integers of size $n \times m$ and $n$ respectively, defining a system of linear equations $A \vec{x} = \vec{b}$
* $\ell$ is a vector of integers of size $m$
* $Q$ is a boolean matrix of size $m \times m$
* $p$ is an integer in the interval $\{0, \dots, 7\}$
* $m$ is an integer

All of these correspond to the following methods of the `State` class:
* `.is_zero()` returns a boolean corresponding to whether the state is the zero state $ $
* `.magnitude()` corresponds to $m$
* `.phase()` corresponds to $p$
* `.affine_part()` returns a tuple with the numpy matrices corresponding to $A$ and $b$
* `.phase_polynomial_matrix()` returns a numpy matrix corresponding to $Q$
* `.lin_part()` returns a numpy vector corresponding to $\ell$


## Working with Maps

Let's start with a few simple examples! There are a couple of pre-defined maps:

In [ ]:
x = Q.x_map() # Pauli X
z = Q.z_map() # Pauli Z
y = Q.y_map() # Pauli Y
i = Q.i_map() # i scalar
j = Q.j_map() # j scalar (8th root of unity)
h = Q.h_map() # Hadamard gate
s = Q.s_map() # S gate
cx = Q.cx_map() # CNOT
cz = Q.cz_map() # CZ
swap = Q.swap_map() # SWAP
zero_post = Q.zero_post() # Projector on the zero state
zero_prep = Q.zero_prep() # Prepares the zero state

We can compose maps using the `*` operator or the `compose` function. We can also tensor maps using the 'tensor' function and check for equality using the `==` operator:

In [ ]:
swap * Q.tensor(x, z) * swap == Q.tensor(z, x)

True

While the size of a state is just a single number (`.n()`), the size of a map consists of two numbers: the number of input wires and the number of output wires (`.in_wires()` and `.out_wires()`). For example, the Paulis have 1 input wire and 1 output wire, whereas the scalar map j has 0 input wires and 1 output wire. These numbers, of course, do not have to be the same, the `zero_prep` map has 0 input wires and 1 output wire and the `zero_post` map has 1 input wire and 0 output wires:

In [ ]:
print('X', x.in_wires(), x.out_wires())
print('j', j.in_wires(), j.out_wires())
print('zero_prep', zero_prep.in_wires(), zero_prep.out_wires())
print('zero_post', zero_post.in_wires(), zero_post.out_wires())

X 1 1
j 0 0
zero_prep 0 1
zero_post 1 0


Of course, two maps `A` and `B` can only be composed if `A.out_wires() == B.in_wires()`.

While the maps listed above suffice to describe any Clifford map via tensoring and composition, it is sometimes more convenient to use the methods of the `Map` class. The following map is a projector onto the $+1$ eigenspace of the $ZZ$ operator:

In [ ]:
zz_filter = (
    Q.id_map(2) # identity map on 2 qubits
    .prep_zero(2) # prepares the zero state on the new 3 qubit
    .cx(0, 2) # CNOT with the first qubit as control and the third as target
    .cx(1, 2) # CNOT with the second qubit as control and the third as target
    .post_zero(2) # postselects on the third qubit being in the state $|0\rangle$
)

A comprehensive list of `Map` methods is given below:
* `.s(pos)` applies an S gate to the qubit at position `pos`
* `.h(pos)` applies a Hadamard gate to the qubit at position `pos`
* `.x(pos)` applies an X gate to the qubit at position `pos`
* `.z(pos)` applies a Z gate to the qubit at position `pos`
* `.cx(pos0, pos1)` applies a CNOT with the qubit at position `pos0` as control and the qubit at position `pos1` as target
* `.cz(pos0, pos1)` applies a CZ with the qubit at position `pos0` as control and the qubit at position `pos1` as target
* `.swap(pos0, pos1)` applies a SWAP with the qubit at position `pos0` and the qubit at position `pos1`
* `.prep_zero()` appends a qubit in the state $|0\rangle$ to the output wires
* `.prep_zero(pos)` moves all qubits starting from position `pos` one position to the right and appends a qubit in the state $|0\rangle$ at position `pos`
* `.post_zero(pos)` moves all qubits starting from position `pos` one position to the left and removes the qubit that was at position `pos`

all of these methods apply the corresponding operation to the output wires of the map. Analagous methods exist for the `State` class.

## Working with Maps and States


Maps and States are closely related. One can, for example, apply a map to a state using `apply_map`; in the following example, we apply the $ZZ$ filter from the previous section to the uniform superposition over the computational basis states of 2 qubits:

In [ ]:
my_qubits = Q.State(2)
Q.apply_h(my_qubits, 0)
Q.apply_h(my_qubits, 1)

filtered_state = Q.apply_map(zz_filter, my_qubits)
display(Q.to_latex(filtered_state))

<IPython.core.display.Math object>

Any state can be converted to a map using the `to_map` function. This produces a state preparation map, which has no input and the state's size output wires:

In [ ]:
prep_qubits = Q.to_map(my_qubits)
print(prep_qubits.in_wires(), prep_qubits.out_wires())

0 2


One can also retrieve the [Choi State](https://en.wikipedia.org/wiki/Choi%E2%80%93Jamio%C5%82kowski_isomorphism) of a map using the `choi_state` function. Here's the Choi state of the Pauli $X$ gate:

In [ ]:
x_choi = Q.choi_state(x)
display(Q.to_latex(x_choi))

<IPython.core.display.Math object>

\* Note that for a Clifford map $F$ with $n$ input wires and $m$ output wires, the Choi state has $n + m$ qubits and is of the form:
$$
\sum_{x \in \{0, 1\}^n} |x \rangle \otimes F |x \rangle
$$

## Last Quirks

Both `Map` and `State` are passed by reference, so you can be screwed by something like this:

In [ ]:
a = Q.State(1)
b = a
display(Q.to_latex(b))
Q.apply_x(a, 0)
display(Q.to_latex(b))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

To avoid embarrassment, use the `.clone()` method (available for both `Map` and `State`):

In [ ]:
a = Q.State(1)
b = a.clone()
display(Q.to_latex(b))
Q.apply_x(a, 0)
display(Q.to_latex(b))

<IPython.core.display.Math object>

<IPython.core.display.Math object>